<a href="https://colab.research.google.com/github/flipkast/tortoise-tts/blob/main/tortoise_tts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome to Tortoise! 🐢🐢🐢🐢

Before you begin, I **strongly** recommend you turn on a GPU runtime.

There's a reason this is called "Tortoise" - this model takes up to a minute to perform inference for a single sentence on a GPU. Expect waits on the order of hours on a CPU.

In [1]:
!git clone https://github.com/neonbjb/tortoise-tts.git
%cd tortoise-tts
!pip install -r requirements.txt

Cloning into 'tortoise-tts'...
remote: Enumerating objects: 1229, done.
remote: Counting objects: 100% (478/478), done.
remote: Compressing objects: 100% (459/459), done.
remote: Total 1229 (delta 26), reused 465 (delta 18), pack-reused 751
Receiving objects: 100% (1229/1229), 372.09 MiB | 25.33 MiB/s, done.
Resolving deltas: 100% (202/202), done.
/content/tortoise-tts
     |████████████████████████████████| 4.0 MB 9.7 MB/s 
     |████████████████████████████████| 6.6 MB 37.7 MB/s 
     |████████████████████████████████| 235 kB 51.8 MB/s 
     |████████████████████████████████| 895 kB 39.3 MB/s 
     |████████████████████████████████| 77 kB 6.3 MB/s 
     |████████████████████████████████| 596 kB 46.5 MB/s 
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12082 sha256=a07569c3e0ff17263f8123966a9444afa85dc4c36c0d0dc1c74da1454e31ee96
  Stored in directory: /root/.cache/pip/wheels/f0/fd/1f/3e35ed57e94cd8ced38dd46771f1f0f94f65fec548659ed855
  Created wheel fo

In [2]:
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from api import TextToSpeech
from utils.audio import load_audio, get_voices

# This will download all the models used by Tortoise from the HF hub.
tts = TextToSpeech()

Done.


Done.


Done.


Done.


Done.


Done.
Removing weight norm...


In [3]:
# List all the voices available. These are just some random clips I've gathered
# from the internet as well as a few voices from the training dataset.
# Feel free to add your own clips to the voices/ folder.
%ls voices

angie/   freeman/  lj/      snakes/         train_grace/     william/
daniel/  geralt/   mol/     tom/            train_kennard/
deniro/  halle/    myself/  train_atkins/   train_lescault/
emma/    jlaw/     pat/     train_dotrice/  weaver/


In [47]:
# This is the text that will be spoken.
text = "Remember, you must follow these steps carefully to ensure the account works properly. We do not give refunds, but we will give replacements if found that the fault was on our side and reported within 24 hours of the purchase."

# Here's something for the poetically inclined.. (set text=)
"""
Then took the other, as just as fair,
And having perhaps the better claim,
Because it was grassy and wanted wear;
Though as for that the passing there
Had worn them really about the same,"""

# Pick one of the voices from above
voice = 'snakes'
# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "high_quality"

In [48]:
# Fetch the voice references and forward execute!
voices = get_voices()
cond_paths = voices[voice]
conds = []
for cond_path in cond_paths:
    c = load_audio(cond_path, 22050)
    conds.append(c)

gen = tts.tts_with_preset(text, conds, preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

Generating autoregressive samples..


100%|██████████| 32/32 [08:12<00:00, 15.38s/it]


Computing best candidates using CLVP and CVVP


  0%|          | 0/32 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
100%|██████████| 32/32 [00:42<00:00,  1.32s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 1024/1024 [05:07<00:00,  3.33it/s]


In [ ]:
# You can add as many conditioning voices as you want together. Combining
# clips from multiple voices takes the mean of the latent space for all
# voices. This creates a novel voice that is a combination of the two inputs.
#
# Lets see what it would sound like if Picard and Kirk had a kid with a penchant for philosophy:
conds = []
for v in ['pat', 'william']:
  cond_paths = voices[v]
  for cond_path in cond_paths:
      c = load_audio(cond_path, 22050)
      conds.append(c)

gen = tts.tts_with_preset("They used to say that if man was meant to fly, he’d have wings. But he did fly. He discovered he had to.", conds, preset)
torchaudio.save('captain_kirkard.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('captain_kirkard.wav')

Generating autoregressive samples..


 22%|██▏       | 7/32 [00:46<02:41,  6.47s/it]